In [3]:
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
# UCI HAR 데이터 로드 함수
def load_uci_har_data():
    X_train = pd.read_csv("E:/dataset/HAR/UCI-HAR/train/X_train.txt", delim_whitespace=True, header=None)
    Y_train = pd.read_csv("E:/dataset/HAR/UCI-HAR/train/y_train.txt", delim_whitespace=True, header=None)
    X_test = pd.read_csv("E:/dataset/HAR/UCI-HAR/test/X_test.txt", delim_whitespace=True, header=None)
    Y_test = pd.read_csv("E:/dataset/HAR/UCI-HAR/test/y_test.txt", delim_whitespace=True, header=None)
    
    return X_train.values, Y_train.values.ravel(), X_test.values, Y_test.values.ravel()

In [5]:
# 저역 통과 필터 적용 함수 (노이즈 제거)
def low_pass_filter(data, cutoff=0.1, fs=50, order=2):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    
    padlen = max(3 * order, 9)
    if len(data) < padlen:
        return data  # 데이터 길이가 padlen보다 작으면 필터 적용 없이 반환
    
    return filtfilt(b, a, data)

In [6]:
# 특징 추출 함수
def extract_features(X, feature_indices):
    features = {}

    # 선택한 센서 데이터 컬럼 필터링
    z_accel = low_pass_filter(X[:, feature_indices['body_acc_z']])
    y_accel = low_pass_filter(X[:, feature_indices['body_acc_y']])
    gyro_x = low_pass_filter(X[:, feature_indices['body_gyro_x']])
    gyro_y = low_pass_filter(X[:, feature_indices['body_gyro_y']])

    # Z축 가속도 변화량 (계단 오를 때 증가, 내릴 때 감소)
    features['z_accel_change'] = np.max(z_accel) - np.min(z_accel)

    # Y축 가속도의 변화량 (계단 오를 때 변화 크고, 내릴 때 변화 적음)
    features['y_accel_var'] = np.var(y_accel)

    # X축 자이로스코프 평균값 (피치 변화 분석)
    features['gyro_x_mean'] = np.mean(gyro_x)

    # Y축 자이로스코프 분산 (계단 내릴 때 균형 조절을 위한 변화량이 큼)
    features['gyro_y_var'] = np.var(gyro_y)

    return features

In [7]:
# 주요 센서 데이터 컬럼 인덱스
feature_indices = {
    'body_acc_z': 2,
    'body_acc_y': 1,
    'body_gyro_x': 39,
    'body_gyro_y': 40
}

In [8]:
import numpy as np

# UCI HAR 데이터 로드
X_train, Y_train, X_test, Y_test = load_uci_har_data()

# Y_train과 Y_test의 고유값 확인 (디버깅)
print("Unique values in Y_train:", np.unique(Y_train))
print("Unique values in Y_test:", np.unique(Y_test))

# 걷기(1), 계단 오르기(2), 계단 내리기(3) 데이터만 필터링
selected_labels = [1, 2, 3]
train_mask = np.isin(Y_train, selected_labels)
test_mask = np.isin(Y_test, selected_labels)

X_train, Y_train = X_train[train_mask], Y_train[train_mask]
X_test, Y_test = X_test[test_mask], Y_test[test_mask]

# 다시 Y_train과 Y_test의 고유값 확인
print("Filtered Unique values in Y_train:", np.unique(Y_train))
print("Filtered Unique values in Y_test:", np.unique(Y_test))



C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_10688\1286935637.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv("E:/dataset/HAR/UCI-HAR/train/X_train.txt", delim_whitespace=True, header=None)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_10688\1286935637.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  Y_train = pd.read_csv("E:/dataset/HAR/UCI-HAR/train/y_train.txt", delim_whitespace=True, header=None)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_10688\1286935637.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv("E:/dataset/HAR/UCI-HAR/test/X_test.txt", delim_whitespace=True, header=None)


Unique values in Y_train: [1 2 3 4 5 6]
Unique values in Y_test: [1 2 3 4 5 6]
Filtered Unique values in Y_train: [1 2 3]
Filtered Unique values in Y_test: [1 2 3]


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_10688\1286935637.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  Y_test = pd.read_csv("E:/dataset/HAR/UCI-HAR/test/y_test.txt", delim_whitespace=True, header=None)


In [9]:
import numpy as np

# 레이블 변환 (1,2,3 → 0,1,2)
label_mapping = {1: 0, 2: 1, 3: 2}

# Y_train과 Y_test의 고유값 확인
print("Unique values in Y_train before mapping:", np.unique(Y_train))
print("Unique values in Y_test before mapping:", np.unique(Y_test))

# 변환 수행 (None 값 방지)
Y_train = np.vectorize(lambda y: label_mapping.get(y, -1))(Y_train)
Y_test = np.vectorize(lambda y: label_mapping.get(y, -1))(Y_test)

# 변환 후 -1 값이 있는지 확인
if np.any(Y_train == -1) or np.any(Y_test == -1):
    print("Warning: Unexpected label found in Y_train or Y_test after filtering!")

# 데이터 타입 변환
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# 변환 후 고유값 확인
print("Transformed Unique values in Y_train:", np.unique(Y_train))
print("Transformed Unique values in Y_test:", np.unique(Y_test))



Unique values in Y_train before mapping: [1 2 3]
Unique values in Y_test before mapping: [1 2 3]
Transformed Unique values in Y_train: [0 1 2]
Transformed Unique values in Y_test: [0 1 2]


In [10]:
# 피처 추출
features_train = [extract_features(X_train[i].reshape(1, -1), feature_indices) for i in range(X_train.shape[0])]
features_test = [extract_features(X_test[i].reshape(1, -1), feature_indices) for i in range(X_test.shape[0])]

# 데이터프레임 변환
df_train = pd.DataFrame(features_train)
df_test = pd.DataFrame(features_test)

# 정규화
scaler_features = StandardScaler()
X_train_features = scaler_features.fit_transform(df_train)
X_test_features = scaler_features.transform(df_test)

scaler_raw = StandardScaler()
X_train_raw = scaler_raw.fit_transform(X_train)
X_test_raw = scaler_raw.transform(X_test)

# 레이블을 원-핫 인코딩
Y_train = to_categorical(Y_train, num_classes=3)
Y_test = to_categorical(Y_test, num_classes=3)

In [ ]:
# Feature-based 모델 (DNN 1)
input_features = Input(shape=(X_train_features.shape[1],))
x1 = Dense(64, activation='relu')(input_features)
x1 = Dense(32, activation='relu')(x1)
x1 = Dense(16, activation='relu')(x1)

# Raw Data 모델 (DNN 2)
input_raw = Input(shape=(X_train_raw.shape[1],))
x2 = Dense(128, activation='relu')(input_raw)
x2 = Dense(64, activation='relu')(x2)
x2 = Dense(32, activation='relu')(x2)

# 모델 병합
merged = Concatenate()([x1, x2])
merged = Dense(32, activation='relu')(merged)
output = Dense(3, activation='softmax')(merged)

# 최종 모델 생성
final_model = Model(inputs=[input_features, input_raw], outputs=output)
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
final_model.fit([X_train_features, X_train_raw], Y_train, epochs=50, batch_size=32, validation_data=([X_test_features, X_test_raw], Y_test))

# 모델 평가
loss, accuracy = final_model.evaluate([X_test_features, X_test_raw], Y_test)
print(f"앙상블 딥러닝 모델 정확도: {accuracy:.4f}")

Epoch 1/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8079 - loss: 0.4515 - val_accuracy: 0.9063 - val_loss: 0.3283
Epoch 2/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9928 - loss: 0.0184 - val_accuracy: 0.8868 - val_loss: 0.4131
Epoch 3/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9946 - loss: 0.0191 - val_accuracy: 0.9106 - val_loss: 0.3058
Epoch 4/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9987 - loss: 0.0056 - val_accuracy: 0.9329 - val_loss: 0.2880
Epoch 5/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9964 - loss: 0.0080 - val_accuracy: 0.9315 - val_loss: 0.3016
Epoch 6/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 2.7775e-04 - val_accuracy: 0.9293 - val_loss: 0.3077
Epoch 7/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 1.1194e-04 - val_accuracy: 0.9301 - val_loss: 0.3178
Epoch 8/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 1.0286e-04 - val_

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# 모델 예측 수행
y_pred = final_model.predict([X_test_features, X_test_raw])

# One-hot encoded -> 정수형 레이블 변환
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(Y_test, axis=1)

# Confusion Matrix 출력
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)
print("Confusion Matrix:\n", conf_matrix)

# F1 Score 및 상세 평가 출력
report = classification_report(y_true_labels, y_pred_labels, target_names=['Walking', 'Stair Up', 'Stair Down'])
print("\nClassification Report:\n", report)
